In [1]:
def showImage(img, imgName): 
    
    height = 500
    heightDiff = ((img.shape[0] - height)/max(img.shape[0], height))
    width = img.shape[1] - img.shape[1]*heightDiff
    width = int(np.round(width))
    dim = (width, height)  
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    cv2.imshow(f"{imgName}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [2]:
import face_recognition
import glob
import numpy as np
import cv2
import pandas as pd
from datetime import datetime


def encodeFeatures(imgsPath):
    
    encodedImgList = []
    for imgPath in glob.glob(imgsPath):
        imgPath = imgPath.replace("\\",'/')
        imgUnique = face_recognition.load_image_file(imgPath)
        imgUnique = cv2.cvtColor(imgUnique,cv2.COLOR_BGR2RGB)
        encodeUnique = face_recognition.face_encodings(imgUnique)[0]
        encodedImgList.append([encodeUnique, imgPath[15:-4]])
    
    return encodedImgList



def faceRecognition(path, encodedImgList, time = 0, date = 0, attendanceDF = None, video = False):
    
    if video == True:
        imgTest = cv2.cvtColor(path,cv2.COLOR_BGR2RGB)
    else :     
        imgTest = face_recognition.load_image_file(path)
        imgTest = cv2.cvtColor(imgTest,cv2.COLOR_BGR2RGB)
        
    faceLocTest = face_recognition.face_locations(imgTest)
    encodedImgTest = face_recognition.face_encodings(imgTest)
    totalFaceDetected = len(faceLocTest)

    nameRecognized = ''
    faceRecognized = 0
    
    thickness = imgTest.shape[0]//400 + 1
    
    for encodedImg in encodedImgList:

        for i in range(totalFaceDetected):
            
            result = face_recognition.compare_faces([encodedImg[0]],encodedImgTest[i])
            if result[0] == True:
                
                
                if video == True:
                    attendanceDF[date][attendanceDF["Name"]== encodedImg[1]] = f'P, {time}'
                    
                    
                tempFaceLocTest = faceLocTest[i]
                height = tempFaceLocTest[2] - tempFaceLocTest[0]
                font = height/70
                faceRecognized += 1
                nameRecognized = f"{encodedImg[1]} : {time}"
                
                #faceDis = face_recognition.face_distance([encodedImg[0]],encodedImgTest[i])
                cv2.rectangle(imgTest,(tempFaceLocTest[3],tempFaceLocTest[0]),(tempFaceLocTest[1],tempFaceLocTest[2]),(0,255,0),thickness)
                cv2.putText(imgTest,f'{nameRecognized}',(tempFaceLocTest[3],tempFaceLocTest[0]-thickness*3),cv2.FONT_HERSHEY_COMPLEX,font,(0,255,0),thickness)


        if totalFaceDetected == faceRecognized:
            break    
    if video == True:        
        return imgTest , nameRecognized, attendanceDF         
    else:
        showImage(imgTest, f'{nameRecognized}')              

In [3]:
imgsPath = "Image Database/*"
encodedImgList = encodeFeatures(imgsPath)

In [4]:
now = datetime.now()
date = now.strftime('%d-%b-%Y')

attendanceDF = pd.read_excel('Attendance Sheet.xlsx', index_col = 0)

if "Name" not in attendanceDF.columns: 
    attendanceDF["Name"] = ""
    
if date not in attendanceDF.columns:
    attendanceDF[f'{date}'] = ""

for encodedImg in encodedImgList:
    if encodedImg[1] not in attendanceDF['Name'].to_list():
        attendanceDF.loc[len(attendanceDF.index), "Name"] = encodedImg[1]
    
attendanceDF = attendanceDF.sort_values("Name")    

In [5]:
vid = cv2.VideoCapture(0)
  
while(True):

    ret, frame = vid.read()
    
    now = datetime.now()
    date = now.strftime('%d-%b-%Y')
    time = now.strftime('%I:%M %p')
    frame , nameRecognized, attendanceDF = faceRecognition(frame, encodedImgList, time, date, attendanceDF, video = True)

    cv2.imshow('Image Recognizing', frame[:,:,::-1])

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

attendanceDF.to_excel("Attendance Sheet.xlsx")

In [6]:
attendanceDF

,Name,29-Aug-2021
0,Aman Kumar,"P, 10:20 AM"
1,Bill Gates,"P, 10:20 AM"
2,Elon Musk,"P, 10:17 AM"
3,Jack Ma,"P, 10:19 AM"
4,Mark Zuckerberg,NaN


In [6]:
# For image if video is not available
path = 'Image Test/Jack Elon.jpg'
faceRecognition(path, encodedImgList, video = False)